In [27]:
import pandas as pd
import os

from pandas_profiling import ProfileReport

In [28]:
file_path =  os.path.abspath(path.join('' ,"../..")) + '/Data/DHS-PROCESSED-CLEAN.csv'

In [29]:
df = pd.read_csv(file_path)

In [30]:
profile = ProfileReport(df, title="Pandas Profiling Report")

In [33]:
#profile.to_file("DHS_EDA_report.html")

## Modeling

In [38]:
cols_drop = ['DHSCLUST', 'Wealth', 'Wealth Index','ADM1NAME', 'LATNUM', 'LONGNUM', 'geometry']

X = df.drop(columns=cols_drop)
y = df[['Wealth Index']]

In [51]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [44]:
X_tr,X_te,y_tr,y_te = train_test_split(X,y)

In [52]:
scaler = StandardScaler()

In [ ]:

scaler.fit_transform(df)